# Day 13: Sales Performance Dashboard 📊

**Objective:** Build a quick matrix to identify top sales reps and category specialists.
**Tech:** Pandas `pivot_table` for reshaping and Styler for the UI.

---

## 1. Data Setup
Simulating a transaction log since I don't have a live DB connection here.
Generating 1,500 rows with a Pareto-like distribution (few large sales, many small ones).

In [4]:
import pandas as pd
import numpy as np

# Setting seed so numbers don't change every time I run this
np.random.seed(42)

# Config
n_transactions = 1500
sales_reps = ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank']
categories = ['Software', 'Consulting', 'Hardware', 'Maintenance']

# Simulating the Data
df_sales = pd.DataFrame({
    'Transaction_ID': range(1, n_transactions + 1),
    'Date': pd.date_range(start='2024-01-01', periods=n_transactions, freq='h'),
    'Sales_Rep': np.random.choice(sales_reps, n_transactions),
    'Category': np.random.choice(categories, n_transactions),
    # Using exponential distribution to mimic real sales (lots of small deals, few whales)
    'Amount': np.round(np.random.exponential(scale=1000, size=n_transactions) + 100, 2)
})

print(f"Data generated. Rows: {df_sales.shape[0]}")
df_sales.head()

Data generated. Rows: 1500


,Transaction_ID,Date,Sales_Rep,Category,Amount
0,1,2024-01-01 00:00:00,David,Consulting,3419.31
1,2,2024-01-01 01:00:00,Eva,Consulting,385.79
2,3,2024-01-01 02:00:00,Charlie,Maintenance,243.64
3,4,2024-01-01 03:00:00,Eva,Hardware,642.28
4,5,2024-01-01 04:00:00,Eva,Software,1662.26


## 2. Aggregating Data (The Heavy Lifting)
Using `pivot_table` instead of `groupby` because I need a matrix view (Reps x Categories).
This makes it easier for stakeholders to compare performance side-by-side.

In [5]:
# Creating the matrix
# Rows: Reps | Cols: Categories | Values: Sum of Sales
performance_matrix = df_sales.pivot_table(
    values='Amount',
    index='Sales_Rep',
    columns='Category',
    aggfunc='sum',
    fill_value=0 # Important: If a rep sold nothing in a category, put 0, not NaN
)

# Adding a 'Total' column to sort by best overall performer later
performance_matrix['Total_Revenue'] = performance_matrix.sum(axis=1)

# Sorting so the best reps are at the top
performance_matrix = performance_matrix.sort_values(by='Total_Revenue', ascending=False)

# Quick sanity check
display(performance_matrix)

Category,Consulting,Hardware,Maintenance,Software,Total_Revenue
Sales_Rep,,,,,
Alice,68339.54,61345.92,101702.82,66076.80,297465.08
Bob,91590.13,52881.37,75326.18,57628.91,277426.59
Frank,65057.81,69797.20,58885.36,82859.25,276599.62
Charlie,83613.26,60300.62,64419.77,67444.85,275778.50
Eva,78619.16,54982.72,77357.44,62635.92,273595.24
David,62193.63,55132.48,57065.91,94759.18,269151.20


## 3. Dashboard Styling
Raw numbers are hard to scan.
Adding a heatmap to highlight category specialists and data bars for total revenue.

In [6]:
# Formatting config
currency_fmt = "${:,.2f}"

# Applying the style
dashboard = (performance_matrix.style
    .format(currency_fmt)                                   # Currency format
    .background_gradient(cmap='Greens', subset=categories)  # Darker green = Higher sales in that category
    .bar(subset=['Total_Revenue'], color='#d65f5f')         # Bar chart for the total column
    .set_caption("FY2024 Sales Performance Matrix")
)

display(dashboard)

Category,Consulting,Hardware,Maintenance,Software,Total_Revenue
Sales_Rep,,,,,
Alice,"$68,339.54","$61,345.92","$101,702.82","$66,076.80","$297,465.08"
Bob,"$91,590.13","$52,881.37","$75,326.18","$57,628.91","$277,426.59"
Frank,"$65,057.81","$69,797.20","$58,885.36","$82,859.25","$276,599.62"
Charlie,"$83,613.26","$60,300.62","$64,419.77","$67,444.85","$275,778.50"
Eva,"$78,619.16","$54,982.72","$77,357.44","$62,635.92","$273,595.24"
David,"$62,193.63","$55,132.48","$57,065.91","$94,759.18","$269,151.20"


## 4. Key Findings

1.  **Top Performer:** Check the `Total_Revenue` column. The longest bar indicates the MVP.
2.  **Specialization:** Look at the heatmap.
    * Some reps might be "Hardware" heavy but weak in "Consulting".
    * This helps in assigning the right leads to the right person.